# Descriptions

The notebook demonstrates how to use `ImClient`.

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Imports

In [3]:
import logging

import pandas as pd

import core.config as cconfig
import core.finance as cofinanc
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import im_v2.ccxt.data.client as icdcl
import im_v2.common.data.client as icdc
import im_v2.common.db.db_utils as imvcddbut
import im_v2.common.universe as ivcu

/app/amp/core/statistics/random_samples.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
log_level = logging.INFO
hdbg.init_logger(verbosity=log_level)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO  > cmd='/venv/lib/python3.9/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-7560fff5-7ca3-435a-963b-9784515151a3.json'
INFO  # Git
  branch_name='CmampTask8935_Prepare_crypto.com_resampled_data_to_1_min'
  hash='af02ed974'
  # Last commits:
    * af02ed974 jsmerix  Update docs with crypto.com reference                             (   7 hours ago) Tue Jul 9 09:31:26 2024  (HEAD -> CmampTask8935_Prepare_crypto.com_resampled_data_to_1_min, origin/CmampTask8935_Prepare_crypto.com_resampled_data_to_1_min)
    * 0182aefb4 jsmerix  Add resampling DAG                                                (  21 hours ago) Mon Jul 8 19:39:21 2024           
    * 3d921ae5f Shayan   AlertManager optimization (#8912)                                 (  21 hours ago) Mon Jul 8 19:36:26 2024  (origin/master, origin/HEAD, master)
# Machine info
  system=Linux
  node name=a3ebf2999eba
  release=5.15.0-1058-aws
  version=#64~20.04.1-Ubuntu SMP Tue Apr 9 11:12:27 UTC 2024
 

# Config

In [ ]:
config = {
    "universe": {
        "vendor": "CCXT",
        "mode": "trade",
        "version": "v7.4",
        "as_full_symbol": True,
    },
    "start_timestamp": pd.Timestamp("2023-09-11T00:00:00", tz="UTC"),
    "end_timestamp": pd.Timestamp("2023-09-11T04:00:00", tz="UTC"),
    "columns": None,
    "filter_data_mode": "assert",
    "ohlcv_parquet_config": {
        "vendor": "ccxt",
        "universe_version": "v7.4",
        "root_dir": "s3://cryptokaizen-data/v3",
        "partition_mode": "by_year_month",
        "dataset": "ohlcv",
        "contract_type": "futures",
        "data_snapshot": "",
        "aws_profile": "ck",
        "resample_1min": False,
        "version": "v1_0_0",
        "download_universe_version": "v7_3",
        "tag": "downloaded_1min",
    },
    "bid_ask_parquet_config": {
        "vendor": "ccxt",
        "universe_version": "v7.4",
        "root_dir": "s3://cryptokaizen-data-test/v3",
        "partition_mode": "by_year_month",
        "dataset": "bid_ask",
        "contract_type": "futures",
        "data_snapshot": "",
        "version": "v1_0_0",
        "download_universe_version": "v7",
        "tag": "resampled_1min",
        "aws_profile": "ck",
    },
    "ohlcv_db_config": {
        "universe_version": "infer_from_data",
        "db_connection": imvcddbut.DbConnectionManager.get_connection("preprod"),
        "table_name": "ccxt_ohlcv_futures",
        "resample_1min": False,
    },
}
config = cconfig.Config().from_dict(config)
print(config)

# CCXT Binance 1-minute futures data

In [ ]:
full_symbols = ivcu.get_vendor_universe(**config["universe"])
_LOG.info("Full symbols number=%s", len(full_symbols))
full_symbols[:5]

## OHLCV Parquet

In [ ]:
ohlcv_im_client = icdc.HistoricalPqByCurrencyPairTileClient(
    **config["ohlcv_parquet_config"]
)

In [ ]:
ohlcv_data = ohlcv_im_client.read_data(
    full_symbols,
    config["start_timestamp"],
    config["end_timestamp"],
    config["columns"],
    config["filter_data_mode"],
)
ohlcv_data.head()

## Bid/ask Parquet

In [ ]:
bid_ask_im_client = icdc.HistoricalPqByCurrencyPairTileClient(
    **config["bid_ask_parquet_config"]
)

In [ ]:
bid_ask_data = bid_ask_im_client.read_data(
    full_symbols,
    config["start_timestamp"],
    config["end_timestamp"],
    config["columns"],
    config["filter_data_mode"],
)
bid_ask_data.head()

## OHLCV Database

In [ ]:
ohlcv_db_im_client = icdcl.CcxtSqlRealTimeImClient(**config["ohlcv_db_config"])

In [ ]:
ohlcv_db_data = ohlcv_db_im_client.read_data(
    full_symbols,
    config["start_timestamp"],
    config["end_timestamp"],
    config["columns"],
    config["filter_data_mode"],
)
ohlcv_db_data.head()

# Mock `ImClient`

In [ ]:
df = cofinanc.get_MarketData_df6(full_symbols)
df.head()

In [ ]:
dataframe_im_client = icdc.DataFrameImClient(
    df,
    full_symbols,
)

In [ ]:
start_timestamp = pd.Timestamp("2000-01-01 09:35:00-05:00")
end_timestamp = pd.Timestamp("2000-01-01 10:31:00-05:00")
#
ohlcv_from_df_data = dataframe_im_client.read_data(
    full_symbols,
    start_timestamp,
    end_timestamp,
    config["columns"],
    config["filter_data_mode"],
)
ohlcv_from_df_data.head()

# CCXT Crypto.com 1-minute bid/ask futures data

In [5]:
config = {
    "universe": {
        "vendor": "CCXT",
        "mode": "trade",
        "version": "v7.5",
        "as_full_symbol": True,
    },
    # This is roughly the span of the dataset but there will be gaps for sure.
    "start_timestamp": pd.Timestamp("2024-01-22T00:00:00", tz="UTC"),
    "end_timestamp": pd.Timestamp("2024-07-08T23:59:00", tz="UTC"),
    "columns": None,
    "filter_data_mode": "assert",
    "bid_ask_parquet_config": {
        "vendor": "ccxt",
        "universe_version": "v7.5",
        "root_dir": "s3://cryptokaizen-data.preprod/tokyo/v3",
        "partition_mode": "by_year_month",
        "dataset": "bid_ask",
        "contract_type": "futures",
        "data_snapshot": "",
        "version": "v2_0_0",
        "download_universe_version": "v7_5",
        "tag": "resampled_1min",
        "aws_profile": "ck",
    },
}
config = cconfig.Config().from_dict(config)
print(config)

universe: 
  vendor: CCXT
  mode: trade
  version: v7.5
  as_full_symbol: True
start_timestamp: 2024-01-22 00:00:00+00:00
end_timestamp: 2024-07-08 23:59:00+00:00
columns: None
filter_data_mode: assert
bid_ask_parquet_config: 
  vendor: ccxt
  universe_version: v7.5
  root_dir: s3://cryptokaizen-data.preprod/tokyo/v3
  partition_mode: by_year_month
  dataset: bid_ask
  contract_type: futures
  data_snapshot: 
  version: v2_0_0
  download_universe_version: v7_5
  tag: resampled_1min
  aws_profile: ck


In [6]:
full_symbols = ivcu.get_vendor_universe(**config["universe"])
# Filter crypto.com symbols
full_symbols = [fs for fs in full_symbols if fs.startswith("crypto")]
full_symbols

['cryptocom::BTC_USD', 'cryptocom::ETH_USD']

In [7]:
bid_ask_im_client = icdc.HistoricalPqByCurrencyPairTileClient(
    **config["bid_ask_parquet_config"]
)

In [8]:
bid_ask_data = bid_ask_im_client.read_data(
    full_symbols,
    config["start_timestamp"],
    config["end_timestamp"],
    config["columns"],
    config["filter_data_mode"],
)
bid_ask_data.head()

,full_symbol,level_1.bid_price.open,level_1.bid_size.open,level_1.ask_price.open,level_1.ask_size.open,level_1.bid_ask_midpoint.open,level_1.half_spread.open,level_1.log_size_imbalance.open,level_1.bid_price.close,level_1.bid_size.close,level_1.ask_price.close,level_1.ask_size.close,level_1.bid_ask_midpoint.close,level_1.half_spread.close,level_1.log_size_imbalance.close,level_1.bid_price.high,level_1.bid_size.max,level_1.ask_price.high,level_1.ask_size.max,level_1.bid_ask_midpoint.max,level_1.half_spread.max,level_1.log_size_imbalance.max,level_1.bid_price.low,level_1.bid_size.min,level_1.ask_price.low,level_1.ask_size.min,level_1.bid_ask_midpoint.min,level_1.half_spread.min,level_1.log_size_imbalance.min,level_1.bid_price.mean,level_1.bid_size.mean,level_1.ask_price.mean,level_1.ask_size.mean,level_1.bid_ask_midpoint.mean,level_1.half_spread.mean,level_1.log_size_imbalance.mean,level_1.bid_ask_midpoint_var.100ms,level_1.bid_ask_midpoint_autocovar.100ms,level_1.log_size_imbalance_var.100ms,level_1.log_size_imbalance_autocovar.100ms,knowledge_timestamp
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-01-23 11:29:00+00:00,cryptocom::BTC_USD,38827.50,0.060,38830.80,0.0350,38829.150,1.650,0.538996,38819.00,0.009,38821.20,0.1700,38820.100,1.100,-2.938574,38834.90,0.3750,38836.40,0.4600,38835.650,3.450,2.212973,38813.60,0.0032,38815.40,0.0100,38814.550,0.050,-3.688879,38823.636183,0.125929,38824.124851,0.196511,38823.880517,0.244334,-0.567412,343.797500,6.997500,1032.046499,871.614512,2024-07-08 16:05:31.432740+00:00
2024-01-23 11:29:00+00:00,cryptocom::ETH_USD,2225.20,0.800,2225.21,0.1572,2225.205,0.005,1.627093,2223.46,0.300,2223.51,1.5000,2223.485,0.025,-1.609438,2225.35,4.2197,2225.37,5.5000,2225.355,0.105,3.101093,2223.31,0.0500,2223.32,0.0675,2223.315,0.005,-3.178054,2224.303499,1.637590,2224.328251,1.473110,2224.315875,0.012376,0.099942,2.225450,-0.006650,628.014035,504.645611,2024-07-08 16:05:31.432740+00:00
2024-01-23 11:30:00+00:00,cryptocom::BTC_USD,38819.00,0.009,38821.20,0.1700,38820.100,1.100,-2.938574,38775.30,0.009,38787.20,0.0039,38781.250,5.950,0.836248,38826.00,0.4638,38826.30,0.4600,38826.050,5.950,3.081675,38775.30,0.0039,38784.10,0.0011,38779.700,0.050,-2.938574,38808.575000,0.124053,38809.357167,0.141019,38808.966083,0.391083,-0.096310,507.042500,43.625000,1200.530356,1034.071692,2024-07-08 16:05:31.432740+00:00
2024-01-23 11:30:00+00:00,cryptocom::ETH_USD,2223.46,0.300,2223.51,1.5000,2223.485,0.025,-1.609438,2221.67,0.400,2222.33,0.0675,2222.000,0.330,1.779337,2224.01,5.5000,2224.09,5.5000,2224.050,0.330,1.927892,2221.66,0.0500,2222.01,0.0675,2221.835,0.005,-2.772589,2223.210367,1.543130,2223.252233,1.706554,2223.231300,0.020933,-0.168377,2.045125,0.009825,916.115778,796.514091,2024-07-08 16:05:31.432740+00:00
2024-01-23 11:31:00+00:00,cryptocom::BTC_USD,38775.80,0.010,38787.30,0.0039,38781.550,5.750,0.941609,38853.00,0.120,38853.10,0.2150,38853.050,0.050,-0.583146,38856.00,0.3394,38856.20,1.1309,38856.050,14.600,4.081441,38750.00,0.0030,38767.20,0.0030,38758.600,0.050,-4.009661,38803.725833,0.095184,38808.664167,0.067639,38806.195000,2.469167,0.407146,1458.015000,4.262500,1686.949833,1404.414832,2024-07-08 16:05:31.432740+00:00
